# Practical cross-entropy

DL에서 가장 핵심적이고, 다루기 까다로운 것이 agent인데, 보편적으로 이용되고 있는 ML 접근방식을 사용해서 agent의 복잡한 행동을 어떠한 비선형 함수를 통과시키는 것으로 대체하려고 함  
비선형 함수는 환경으로부터의 관측값을 agent의 입력값으로 보고 그것을 함수를 통과시킨 출력값에 Mapping시키는 역할을 한다.  
비선형 함수들이 만들어내는 출력값들은 가치기반 혹은, 정책기반 방법에 의존한다.  
교차엔트로피 방법은 정책기반 방법으로, 우리가 사용할 비선형함수(신경망)는 정책을 출력값으로 내놓는다.  
정책은 기본적으로 agent가 취해야만 하는 action을 결정해준다.
<img src="./images/Fig1.png" />
실질적으로 정책은 action들의 확률분포로 나타내서, 분류문제와 유사하게 만든다.

이런 요약은 agent를 좀 더 단순하게 한다. 바로 환경으로부터 관측값을 받아서 네트워크에 전달해주고, action들의 확률분포를 얻은 다음 그것을 랜덤 샘플링하여 취하게 될 action을 선택한다. agent가 action을 취하고 나면 환경으로부터 새로운 관측값과 함께, 보상이 주어진다. 그리고 다시 반복된다.  
이런 action, observation, 보상은 agent의 lifetime동안 에피소드로 묶이는데 에피소드는 아래 그림과 같이 일련의 관측, 행동, 보상들의 연속이다. 그리고 각 에피소드별로 총 누적 보상값을 구할 수 있다.
<img src="./images/Fig2.png" />
교차엔트로피 방법의 핵심은 보상합이 낮은 나쁜 에피소드를 버리고, 더 좋은 에피소드로 학습을 하는 것이다.

1 N번의 에피소드를 플레이한다.  
2 매 에피소드마다 총 보상을 구하고 경계보상값을 설정한다(주로 모든 보상의 일정 percentile로 설정)  
3 경계값보다 작은 보상을 가진 에피소드들을 버린다  
4 남아있는 "엘리트" 에피소드를 이루는데 사용된 관측값, 행동을 가지고 agent를 학습시킨다  
5 만족할 만한 결과가 나올 때까지 다시 1단계 부터 반복한다.

# Cross-entropy method의 이론적 배경
교차 엔트로피 방법은 중요도 샘플링 정리(importance sampling theorem)에 기반하고 있고 수식은 다음과 같다

$$\mathbb{E}_{x\sim p(x)}[H(x)] = \int_{x}^{} p(x)H(x) dx = \int_{x}^{} q(x) \frac{p(x)}{q(x)} H(x) dx = \mathbb{E}_{x\sim q(x)}[\frac{p(x)}{q(x)} H(x)]$$

위 식에서 H(x)는 정책 x와 그 정책의 가능한 모든 확률분포 p(x)로부터 얻어지는 보상값이다. 보상을 극대화하기 위해서 모든 가능한 정책들을 다 찾아보는 것은 무의미하므로, 샘플링 함수 q(x)를 도입해서 p(x)H(x)와 q(x)와의 차이점을 최소화하는 값으로 수렴하도록 한다.

$$KL(p_1(x)||p_2(x)) = \mathbb{E}_{x\sim p_1(x)}\log\frac{p_1(x)}{p_2(x)} = \mathbb{E}_{x\sim p_1(x)}[\log p_1(x)] - \mathbb{E}_{x\sim p_1(x)}[\log p_2(x)]$$

두 확률분포의 차이가 어느정도 되는지 계산하는 메트릭이 바로 Kullback-Leibler(KL) divergence이고 위의 식으로 정의 된다.  
KL의 첫번째 항이 바로 엔트로피이며 첫 번째 항은 최소화 문제에서 빼도 상관없으며, 두번째 항을 교차엔트로피라고 하며 최소화에 사용된다.

$$ q_{i+1}(x) = \underset{q_{i+1}(x)}{\arg\min} [- \mathbb{E}_{x\sim q_i(x)} \frac{p(x)}{q_i(x)} H(x)\log q_{i+1}(x)] $$

매 스텝마다 q를 계속 업데이트 한다. 이게 일반적인 교차 엔트로피 방법이다.

$$ \pi_{i+1}(a|s) = \underset{\pi_{i+1}}{\arg\min} [ - \mathbb{E}_{z\sim \pi_{i}(a|s)}[R(z)\geq \psi_i]\log \pi_{i+1}(a|s) ] $$

우리가 해결할 문제에 적합시키기 위해서 식을 더 단순화하려면, 우선 H(x)를 indicator함수로 변경해준다. indicator함수는 지정된 threshold보다 큰 reward를 가지면 1, 작은 reward를 가지면 0이 되는 함수이다. 위 식처럼 정책함수를 업데이트 하는 형태로 볼 수 있다.  
엄밀하게는 normalization항이 빠졌지만 실질적으로 문제되지는 않는다.